# Open Street Maps Distance Covariates for Kenya #

#### Libraries ####

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import rasterio.mask
import rasterio.plot
import requests
import folium
from shapely.geometry import LineString
from shapely.geometry import Point
from pathlib import Path
import seaborn as sns
from scipy.ndimage import distance_transform_edt

## Custom Utility Function ##

In [ ]:
# I already have a utility function for punching admin size holes out of rasters
def mask_admin(in_path: Path, admin_path: Path, out_path: Path):
    with rasterio.open(in_path) as src:
        admin = gpd.read_file(admin_path).to_crs(src.crs)
        kwargs = {
            'all_touched': True,
            'nodata': np.nan,
        }
        out_image, out_transform = rasterio.mask.mask(
            src, 
            admin.geometry.tolist(),
            **kwargs        
        )
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "nodata": np.nan,
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(out_path, "w", **out_meta) as dest:
        dest.write(out_image)

# Custom Function  - Rasterizes GDF Geometry and Returns Indices #

In [ ]:
def rasterize_and_get_indices(gdf, out_shape, transform):
    # Rasterize the GeoDataFrame
    rasterized = rasterio.features.rasterize(
        [(geom, 1) for geom in gdf.geometry],
        out_shape=out_shape,
        transform=transform,
        fill=0,
        all_touched=True,
        dtype=rasterio.uint8
    )

    # Get indices where rasterized data is not empty or 0
    indices_ones = np.where(rasterized != 0)
    indices_list = np.array(list(zip(indices_ones[1], indices_ones[0])))
    
    return rasterized, indices_list

# Read in Kenya Data #

In [ ]:
# Bulding Density Data
building_density_path = '/mnt/team/rapidresponse/pub/population/data/03-processed-data/building-layers/KEN/2020_q2/full.tif'
with rasterio.open(building_density_path) as src:
    # We don't need anything from this dataset except the metadata about 
    # it's shape and resolution.
    meta = src.meta
    kenya_shape = src.shape
    
# Transform Data
transform = meta['transform']
# Out metadata is exactly the same as our in metadata.
out_meta = meta.copy()
target_crs = out_meta['crs']

# Resolution Data
resolution = transform.a  # You can access matrix elements by a,b,c,d,etc, just check you're getting the value you expect.
x_offset = transform.xoff
y_offset = transform.yoff

# Get the width and height in pixels from the metadata
width = meta['width']
height = meta['height']

# Make arrays of [0, 1, ..., width] and [0, 1, ..., height]
x_ind = np.arange(width)  # Just like python's `range` function, but makes a numpy array, which we can do vector math with.
y_ind = np.arange(height) 

# Here is the magic of numpy. We can write the same expressions and they'll
# work pretty much seamlessly across scalars and vectors and higher dimensional arrays
# (as long as the operations are well defined.
x = resolution * (x_ind + 1/2) + x_offset  # Makes an array where each element is now an x-coordinate of a centroid in our CRS
y = -resolution * (y_ind + 1/2) + y_offset

# River Covariate #

In [ ]:
# Define a list of waterway types to query
waterway_types = ["river"]

for waterway_type in waterway_types:
    # Define the Overpass query for each waterway type
    query = f"""
    [out:json];
    area["ISO3166-1"="KE"][admin_level=2];
    way["waterway"="{waterway_type}"](area);
    out body;
    >;
    out skel qt;
    """
    
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data=query)

    data = response.json()

# Extract all the nodes
nodes = data['elements']

node_positions = {node['id']: (node['lat'], node['lon']) for node in nodes if node['type'] == 'node'}

# Define the query result elements
ways = data['elements']

# Initialize an empty list to store LineString objects
line_strings = []

# Add rivers to the list of LineString objects
for way in ways:
    if 'tags' in way and 'waterway' in way['tags'] and way['tags']['waterway'] == 'river':
        coordinates = [(node_positions[node_id][1], node_positions[node_id][0]) for node_id in way['nodes']]
        line_strings.append(LineString(coordinates))

# Create a GeoDataFrame from the LineString objects
gdf_rivers = gpd.GeoDataFrame(geometry=line_strings, crs='EPSG:4326')

# Now gdf_rivers contains the linestrings representing the rivers
#gdf_rivers = gdf_rivers.to_crs(target_crs)

In [ ]:
gdf_rivers.to_file('/mnt/team/rapidresponse/pub/population/data/01-raw-data/covariates/KEN/bodies-of-water/open-street-maps/rivers.shp')

In [ ]:
# Initialize a map centered on Kenya
m = folium.Map(location=[1.2921, 36.8219], zoom_start=6)

# Define a function to add rivers to the map
def add_rivers_to_map(ways, color):
    for way in ways:
        coordinates = [(node_positions[node_id][0], node_positions[node_id][1]) for node_id in way['nodes']]
        folium.PolyLine(coordinates, color=color).add_to(m)

# Initialize a map centered on Kenya
m = folium.Map(location=[1.2921, 36.8219], zoom_start=6)

# Add rivers to the map
add_rivers_to_map([way for way in ways if 'tags' in way and 'waterway' in way['tags'] and way['tags']['waterway'] == 'river'], color='blue')

# Display the map in the notebook
m


In [ ]:
raster_river, river_indices = rasterize_and_get_indices(gdf_rivers, kenya_shape, transform)

# SciPy Code #

In [ ]:
# Swap 0s and 1s
scipy_raster = np.logical_not(raster_river).astype(int)

In [ ]:
%%time
river_out = distance_transform_edt(scipy_raster)

In [ ]:
# Create a heatmap using Matplotlib
plt.imshow(river_out, cmap='viridis')
# Add colorbar for reference
plt.colorbar()
# Show the plot
plt.show()

In [ ]:
# Use Natural Earth Shape file as place holder
admin0_path = Path('/mnt/share/homes/mfiking/population/gis/population/kenya_admin_files/ken_admbnda_adm0_iebc_20191031.shp')

# Change the data type of the distance array to the same data type as out_meta
river_out = river_out.astype(out_meta['dtype'])

# Create path to store output
river_out_path = '/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/rivers.tif'

# Generate meta data into out_path same meta data as out_meta
with rasterio.open(river_out_path, "w", **out_meta) as dest:
    dest.write(river_out.reshape((1, *river_out.shape)))

# Just overwrite the file we wrote
mask_admin(river_out_path, admin0_path, river_out_path)

# Read the combined raster
with rasterio.open(river_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs

# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='viridis')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Distance')

# Set the title of the plot
plt.title('Open Street Maps Distance to Nearest River')

plt.savefig('/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/plots/rivers.jpg', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
river_out_path = '/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/rivers.tif'

# Log Plot
with rasterio.open(river_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs
    
    # Apply the log transformation
    combined_raster = np.log(1 + combined_raster)
    
# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='viridis')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Log(1 + Distance)')

# Set the title of the plot
plt.title('Open Street Maps Log Distance to Nearest River')

plt.savefig('/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/plots/log_rivers.jpg', dpi=300, bbox_inches='tight')
plt.show()


# Stream Covariate #

In [ ]:
# Define a list of waterway types to query
waterway_types = ["stream"]

for waterway_type in waterway_types:
    # Define the Overpass query for each waterway type
    query = f"""
    [out:json];
    area["ISO3166-1"="KE"][admin_level=2];
    way["waterway"="{waterway_type}"](area);
    out body;
    >;
    out skel qt;
    """
    
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data=query)

    data = response.json()

# Extract all the nodes
nodes = data['elements']

node_positions = {node['id']: (node['lat'], node['lon']) for node in nodes if node['type'] == 'node'}

# Define the query result elements
ways = data['elements']

# Initialize an empty list to store LineString objects
line_strings = []

# Add rivers to the list of LineString objects
for way in ways:
    if 'tags' in way and 'waterway' in way['tags'] and way['tags']['waterway'] == 'stream':
        coordinates = [(node_positions[node_id][1], node_positions[node_id][0]) for node_id in way['nodes']]
        line_strings.append(LineString(coordinates))

# Create a GeoDataFrame from the LineString objects
gdf_streams = gpd.GeoDataFrame(geometry=line_strings, crs='EPSG:4326')

# Now gdf_rivers contains the linestrings representing the rivers
#gdf_streams = gdf_streams.to_crs(target_crs)



In [ ]:
gdf_streams

In [ ]:
gdf_streams.to_file('/mnt/team/rapidresponse/pub/population/data/01-raw-data/covariates/KEN/bodies-of-water/open-street-maps/streams/streams.shp')

In [ ]:
# Initialize a map centered on Kenya
m = folium.Map(location=[1.2921, 36.8219], zoom_start=6)

# Define a function to add rivers to the map
def add_rivers_to_map(ways, color):
    for way in ways:
        coordinates = [(node_positions[node_id][0], node_positions[node_id][1]) for node_id in way['nodes']]
        folium.PolyLine(coordinates, color=color).add_to(m)

# Initialize a map centered on Kenya
m = folium.Map(location=[1.2921, 36.8219], zoom_start=6)

# Add rivers to the map
add_rivers_to_map([way for way in ways if 'tags' in way and 'waterway' in way['tags'] and way['tags']['waterway'] == 'stream'], color='blue')

# Display the map in the notebook
m


In [ ]:
raster_stream, stream_indices = rasterize_and_get_indices(gdf_streams, kenya_shape, transform)
# Swap 0s and 1s
raster_stream = np.logical_not(raster_stream).astype(int)

In [ ]:
%%time
stream_out = distance_transform_edt(raster_stream)

In [ ]:
# Use Natural Earth Shape file as place holder
admin0_path = Path('/mnt/share/homes/mfiking/population/gis/population/kenya_admin_files/ken_admbnda_adm0_iebc_20191031.shp')

# Change the data type of the distance array to the same data type as out_meta
stream_out = stream_out.astype(out_meta['dtype'])

# Create path to store output
stream_out_path = '/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/streams.tif'

# Generate meta data into out_path same meta data as out_meta
with rasterio.open(stream_out_path, "w", **out_meta) as dest:
    dest.write(stream_out.reshape((1, *stream_out.shape)))

# Just overwrite the file we wrote
mask_admin(stream_out_path, admin0_path, stream_out_path)

# Read the combined raster
with rasterio.open(stream_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs

# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='viridis')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Distance')

# Set the title of the plot
plt.title('Open Street Maps Distance to Nearest Stream')

plt.savefig('/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/plots/streams.jpg', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
# Read the combined raster
with rasterio.open(stream_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs

# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='Set1')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Distance')

# Set the title of the plot
plt.title('Open Street Maps Distance to Nearest Stream')
plt.show()


In [ ]:
stream_out_path = '/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/streams.tif'

# Log Plot
with rasterio.open(stream_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs
    
    # Apply the log transformation
    combined_raster = np.log(1 + combined_raster)
    
# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='viridis')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Log(1 + Distance)')

# Set the title of the plot
plt.title('Open Street Maps Log Distance to Nearest Stream')

plt.savefig('/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/plots/log_streams.jpg', dpi=300, bbox_inches='tight')

plt.show()


## Canals Covariate ##

In [ ]:
# Define a list of waterway types to query
waterway_types = ["canal"]

for waterway_type in waterway_types:
    # Define the Overpass query for each waterway type
    query = f"""
    [out:json];
    area["ISO3166-1"="KE"][admin_level=2];
    way["waterway"="{waterway_type}"](area);
    out body;
    >;
    out skel qt;
    """
    
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data=query)

    data = response.json()

# Extract all the nodes
nodes = data['elements']

node_positions = {node['id']: (node['lat'], node['lon']) for node in nodes if node['type'] == 'node'}

# Define the query result elements
ways = data['elements']

# Initialize an empty list to store LineString objects
line_strings = []

# Add rivers to the list of LineString objects
for way in ways:
    if 'tags' in way and 'waterway' in way['tags'] and way['tags']['waterway'] == 'canal':
        coordinates = [(node_positions[node_id][1], node_positions[node_id][0]) for node_id in way['nodes']]
        line_strings.append(LineString(coordinates))

# Create a GeoDataFrame from the LineString objects
gdf_canals = gpd.GeoDataFrame(geometry=line_strings, crs='EPSG:4326')

# Now gdf_rivers contains the linestrings representing the rivers
#gdf_canals = gdf_canals.to_crs(target_crs)



In [ ]:
gdf_canals.to_file('/mnt/team/rapidresponse/pub/population/data/01-raw-data/covariates/KEN/bodies-of-water/open-street-maps/canals/canals.shp')

In [ ]:



# Initialize a map centered on Kenya
m = folium.Map(location=[1.2921, 36.8219], zoom_start=6)

# Define a function to add rivers to the map
def add_rivers_to_map(ways, color):
    for way in ways:
        coordinates = [(node_positions[node_id][0], node_positions[node_id][1]) for node_id in way['nodes']]
        folium.PolyLine(coordinates, color=color).add_to(m)

# Initialize a map centered on Kenya
m = folium.Map(location=[1.2921, 36.8219], zoom_start=6)

# Add rivers to the map
add_rivers_to_map([way for way in ways if 'tags' in way and 'waterway' in way['tags'] and way['tags']['waterway'] == 'canal'], color='blue')

# Display the map in the notebook
m


In [ ]:
raster_canals, canal_indices = rasterize_and_get_indices(gdf_canals, kenya_shape, transform)

In [ ]:
# Swap 0s and 1s
raster_canals = np.logical_not(raster_canals).astype(int)

In [ ]:
%%time
canal_out = distance_transform_edt(raster_canals)

In [ ]:
# Use Natural Earth Shape file as place holder
admin0_path = Path('/mnt/share/homes/mfiking/population/gis/population/kenya_admin_files/ken_admbnda_adm0_iebc_20191031.shp')

# Change the data type of the distance array to the same data type as out_meta
canal_out = canal_out.astype(out_meta['dtype'])

# Create path to store output
canal_out_path = '/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/canals.tif'

# Generate meta data into out_path same meta data as out_meta
with rasterio.open(canal_out_path, "w", **out_meta) as dest:
    dest.write(canal_out.reshape((1, *canal_out.shape)))

# Just overwrite the file we wrote
mask_admin(canal_out_path, admin0_path, canal_out_path)

# Read the combined raster
with rasterio.open(canal_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs

# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='viridis')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Distance')

# Set the title of the plot
plt.title('Open Street Maps Distance to Nearest Canal')

plt.savefig('/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/plots/canals.jpg', dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
# Read the combined raster
with rasterio.open(canal_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs

# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='Set1')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Distance')

# Set the title of the plot
plt.title('Open Street Maps Distance to Nearest Canal')
plt.show()


In [ ]:
canal_out_path = '/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/canals.tif'

# Log Plot
with rasterio.open(canal_out_path) as src:
    combined_raster = src.read(1, masked=True)
    transform = src.transform
    crs = src.crs
    
    # Apply the log transformation
    combined_raster = np.log(1 + combined_raster)
    
# Plot the combined raster
plt.figure(figsize=(10, 10))
plt.imshow(combined_raster, extent=(transform[2], transform[2] + transform[0]*combined_raster.shape[1],
                                   transform[5] + transform[4]*combined_raster.shape[0], transform[5]), cmap='viridis')

# Add a colorbar
cb = plt.colorbar()
cb.set_label('Log(1 + Distance)')

# Set the title of the plot
plt.title('Open Street Maps Log Distance to Nearest Canal')

plt.savefig('/mnt/share/homes/mfiking/population/gis/population/distance_covariates/open_street_maps/waterways/plots/log_canals.jpg', dpi=300, bbox_inches='tight')

plt.show()
